In [74]:
import boto3
import sagemaker
import pandas as pd
from sagemaker import image_uris
from sagemaker  import TrainingInput
from sagemaker.session import Session
from sklearn.datasets import load_iris

In [75]:
X, y = load_iris(return_X_y=True, as_frame=True)
dados = pd.concat([y, X], axis=1)
dados.to_csv("data/dados.csv", header=False, index=False)

In [76]:
bucket = sagemaker.Session().default_bucket()

In [77]:
xgboost_container = sagemaker.image_uris.retrieve("xgboost", "us-east-1", "1.2-1")

In [78]:
role = "arn:aws:iam::885248014373:role/service-role/AmazonSageMaker-ExecutionRole-20210305T230941"

In [79]:
# initialize hyperparameters
hyperparameters = {
    "objective":"multi:softmax",
    "num_round":"2",
    "num_class": 3
}

In [80]:
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          role=role,
                                          hyperparameters=hyperparameters,
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5,
                                          output_path=f"s3://{bucket}")

In [81]:
input_data = sagemaker.Session().upload_data(path="data", bucket=bucket)
input_data

's3://sagemaker-us-east-1-885248014373/data'

In [82]:
train_input = TrainingInput(input_data, content_type="csv")

In [83]:
estimator.fit({'train': train_input})

2021-03-09 16:20:50 Starting - Starting the training job...
2021-03-09 16:21:14 Starting - Launching requested ML instancesProfilerReport-1615306848: InProgress
......
2021-03-09 16:22:28 Starting - Preparing the instances for training......
2021-03-09 16:23:36 Downloading - Downloading input data
2021-03-09 16:23:36 Training - Downloading the training image...
2021-03-09 16:24:07 Uploading - Uploading generated training model[2021-03-09 16:24:03.320 ip-10-0-241-121.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input i

In [85]:
from sagemaker.model_monitor import DataCaptureConfig

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f"s3://{bucket}",
    capture_options=["REQUEST", "RESPONSE"]
)

In [86]:
estimator.deploy(initial_instance_count=1,
                 instance_type="ml.t2.medium", 
                 data_capture_config = data_capture_config)

----------------------!

In [87]:
estimator.

In [88]:
endpointName = "sagemaker-xgboost-2021-03-09-16-27-01-773"

In [89]:
features = pd.DataFrame(X)
features.to_csv("features.csv", header=False, index=False)

In [97]:
request = open("features.csv").read().splitlines()[0]

In [94]:
er

'5.1,3.5,1.4,0.2'

In [ ]:
import boto3

client = boto3.client("sagemaker-runtime")

for line in  er.splitlines():
    response = client.invoke_endpoint(EndpointName=endpointName,
                                      Body=request,
                                     ContentType="csv")

In [ ]:
response["Body"].read().decode("utf-8")